In [1]:
import os
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification/research


In [2]:
# Goto Root directory of the project
os.chdir("../")
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification


## Basic configuration

In [3]:
# Data Ingestion Config Schema/Data Class
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_dataset_zip_file: Path
    unzip_dir: Path

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import create_directories,read_yaml

In [5]:
CONFIG_FILE_PATH

PosixPath('config/config.yaml')

In [6]:
# Import Config Box
from box import ConfigBox

## Configuration updates

In [7]:
# Create Configuration Manager: This has basic information required before staring any module/components

class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config:ConfigBox = read_yaml(config_path)
        self.params:ConfigBox = read_yaml(params_path)

        # Create Artifacts Root dir
        create_directories([self.config.artifacts_root])
    
    # Read Data Ingestion Config
    def get_data_ingestion_config(self)->DataIngestionConfig:
        data_config = self.config.data_ingestion

        # create data root directory
        create_directories([data_config.root_dir])


        # This is the input that will go to the Data Ingestion c omponent
        data_ingestion_config = DataIngestionConfig(
            root_dir= data_config.root_dir,
            source_url= data_config.source_url,
            local_dataset_zip_file=data_config.local_dataset_zip_file,
            unzip_dir=data_config.unzip_dir
        )

        return data_ingestion_config
    





## Components
1. Data Ingestion

In [8]:
import os
import urllib.request as request
from src.textSummarizer.logging import logger
import zipfile

In [9]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_zip(self):
        if not os.path.exists(self.config.local_dataset_zip_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url, 
                filename=self.config.local_dataset_zip_file
            )
            logger.info("File has been downloaded")
        else:
            logger.info("File already exists")

    def extract_zip(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_dataset_zip_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

In [10]:
# Instantiate Configuration Manager
config_manager = ConfigurationManager()
data_ingestion_config=config_manager.get_data_ingestion_config()

data_ingestion = DataIngestion(config=data_ingestion_config)

data_ingestion.download_zip()
data_ingestion.extract_zip()

[2025-04-22 15:32:30,123 : INFO: common: yaml file: config/config.yaml is loaded sucessfully]
[2025-04-22 15:32:30,124 : INFO: common: yaml file: params.yaml is loaded sucessfully]
[2025-04-22 15:32:30,124 : INFO: common: Created directory at: artifacts]
[2025-04-22 15:32:30,125 : INFO: common: Created directory at: artifacts/data_ingestion]
[2025-04-22 15:32:33,468 : INFO: 245471372: File has been downloaded]
